# Create a Perturbation in a Material

Create a perturbation in a material with a specified smooth function. 

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Set perturbation parameters in cell 2.1. (or use default).
1. Click “Run” > “Run All” to run all cells. 
1. Wait for the run to complete (depending on the parameters can take a few min). 
1. Scroll down to view results. 

## Summary
1. Prepare the Environment: Set up the notebook and install packages, preview the input materials
1. Create the Perturbation: Add a smooth perturbation to the material
2. Visualize the Perturbed Material

## Notes

1. For more information, see [Introduction](Introduction.ipynb)


## 1. Prepare the Environment
### 1.1. Set up defect parameters 

In [ ]:
SUPERCELL_MATRIX = [[5, 0, 0], [0, 5, 0], [0, 0, 1]] 

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip
    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages
    await install_packages("", "../../config.yml")

### 1.3. Get input materials
Materials are loaded with `get_data()`. The first material is assigned as substrate and the second as film.

In [ ]:
from mat3ra.made.material import Material
from utils.jupyterlite import get_data

# Get the list of input materials and load them into `materials_in` variable
get_data("materials_in", globals())
materials = list(map(Material, globals()["materials_in"]))

### 1.4. Create and preview Supercell

In [ ]:
from utils.visualize import visualize_materials as visualize
from mat3ra.made.tools.build.supercell import create_supercell

unit_cell = materials[1]
supercell = create_supercell(unit_cell, supercell_matrix=SUPERCELL_MATRIX)
visualize(supercell, repetitions=[1, 1, 1], rotation="0x")

## 2. Create the Perturbation
### 2.1. Set perturbation parameters

In [ ]:
from mat3ra.made.tools.build.perturbation import PerturbationConfiguration, SlabPerturbationBuilder, \
    CellMatchingDistancePreservingSlabPerturbationBuilder, DistancePreservingSlabPerturbationBuilder
from mat3ra.made.tools.utils.perturbation import SineWavePerturbationFunctionHolder, PerturbationFunctionHolder
import sympy as sp

amplitude = 0.05
wavelength = 1
phase = 0
axis = "y"
perturbation_function = SineWavePerturbationFunctionHolder(amplitude=amplitude, wavelength=wavelength, phase=phase,
                                                           axis=axis)


def function() -> sp.Expr:
    x = sp.Symbol("x")
    y = sp.Symbol("y")
    return amplitude * sp.sin(2 * sp.pi * x / wavelength + phase) * sp.sin(2 * sp.pi * y / wavelength)


perturbation_general_function = PerturbationFunctionHolder(function=function(), variables=["x", "y"])
configuration = PerturbationConfiguration(material=supercell, perturbation_function_holder=perturbation_function,
                                          use_cartesian_coordinates=False)

configuration_general = PerturbationConfiguration(material=supercell,
                                                  perturbation_function_holder=perturbation_general_function,
                                                  use_cartesian_coordinates=False)
builder = SlabPerturbationBuilder()

### 2.2. Apply perturbation to the material

In [ ]:
from mat3ra.made.tools.build.perturbation import create_perturbation

material_with_perturbation = create_perturbation(configuration, builder)
material_with_perturbation_general = create_perturbation(configuration_general, builder)

## 3. Visualize the Material

In [ ]:
from utils.visualize import visualize_materials as visualize

visualize([{"material": supercell, "title": "Original material"},
           {"material": material_with_perturbation, "title": f"Material with perturbation"},
           {"material": material_with_perturbation, "title": f"Material with perturbation", "rotation": "-90x"},
           {"material": material_with_perturbation_general, "title": f"Material with general perturbation"},
           {"material": material_with_perturbation_general, "title": f"Material with general perturbation",
            "rotation": "-90x"}
           ])

## 4. Pass data to the outside runtime

In [ ]:
from utils.jupyterlite import set_data

set_data("materials", [material_with_perturbation.to_json(), material_with_perturbation_general.to_json()])